In [1]:
import sys

sys.path.append('/home/jovyan/work')
%load_ext autoreload
%autoreload 2

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
from helpers.logger import Logger
from helpers.vars import DUMPS_PATH, DATA_PATH, PRE_PATH, RES_PATH

### Load general Data and user kind, datetime, title normalization


In [ ]:
column_names = np.loadtxt(f'{DATA_PATH}/mediawiki_history_columns.txt', dtype=str)

In [ ]:
def process_lang_history(lang, column_names, dtypes, path=DUMPS_PATH, ending='tsv.bz2', years=[2018, 2019, 2020]):
    df_lang = pd.DataFrame()
    # quick fix for small wikis
    try:
        return pd.read_csv(f'{path}/{lang}.{ending}', sep='\t', names=list(column_names), dtype=dtypes, warn_bad_lines=True, error_bad_lines=False)
        Logger.instance().info(f'Loaded {lang} in {time.time() - start}')
    except:
        traceback.print_exc()
        Logger.instance().info(f'PROBABLY EXPECTED ERROR: No "all-times" file for {lang}')
                    
    for year in years:
        start = time.time()
        if lang == 'en':
            start = time.time()
            try:
                for month in range(1, 13): # just throw exception when out of bounds here
                    df_lang = pd.concat([df_lang, pd.read_csv(f'{path}/{lang}-{year}-{month:02d}.{ending}', sep='\t', names=list(column_names), dtype=dtypes, warn_bad_lines=True, error_bad_lines=False)])
                    Logger.instance().info(f'Loaded {lang}-{year}-{month:02d} in {time.time() - start}')
            except:
                Logger.instance().info(f'Error when processing {lang}-{year}-{month}')
        else:
            try:
                df_lang = pd.concat([df_lang, pd.read_csv(f'{path}/{lang}-{year}.{ending}', sep='\t', names=list(column_names), dtype=dtypes, warn_bad_lines=True, error_bad_lines=False)])
                Logger.instance().info(f'Loaded {lang}-{year} in {time.time() - start}')
            except:
                traceback.print_exc()
                Logger.instance().info(f'Error when processing {lang}-{year}')
    return df_lang

In [6]:
import swifter
import pendulum
from mw.lib import title as mw_t
import time
import traceback
from pathlib import Path

In [7]:
# we manually set the timezones here instead of using our pre-loading code
# path_lang_codes = f'{DATA_PATH}/lang_code.csv'
#tz_ = dict(pd.read_csv(path_lang_codes).set_index("code").timezone)
tz_ = {'pl': 'Europe/Warsaw',
'cs': 'Europe/Prague',
'uk': 'Europe/Minsk',
'ru': 'Europe/Moscow'}
tz = {code: pendulum.timezone(t) for code, t in tz_.items()}
codes = list(tz.keys())
codes

['pl', 'cs', 'uk', 'ru']

In [8]:
PRE_PATH

'../data/preprocessing'

In [9]:
df_dict = {}
for code in codes:
    df_dict[code] = process_lang_history(code, column_names, path=DUMPS_PATH, dtypes=str)

Traceback (most recent call last):
  File "<ipython-input-5-c1aa7e8a0571>", line 5, in process_lang_history
    return pd.read_csv(f'{path}/{lang}.{ending}', sep='\t', names=list(column_names), dtype=dtypes, warn_bad_lines=True, error_bad_lines=False)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 610, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 462, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 819, in __init__
    self._engine = self._make_engine(self.engine)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 1050, in _make_engine
    return mapping[engine](self.f, **self.options)  # type: ignore[call-arg]
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 1867, in __init__
    self._open_handles(src, kwds)
  File "/opt/conda/lib/py

New logging instance for ../logging//coronawiki.log


09-01 09:17 : INFO : Loaded pl-2018 in 125.28399729728699
09-01 09:19 : INFO : Loaded pl-2019 in 103.07519245147705
09-01 09:21 : INFO : Loaded pl-2020 in 126.96920680999756
Traceback (most recent call last):
  File "<ipython-input-5-c1aa7e8a0571>", line 5, in process_lang_history
    return pd.read_csv(f'{path}/{lang}.{ending}', sep='\t', names=list(column_names), dtype=dtypes, warn_bad_lines=True, error_bad_lines=False)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 610, in read_csv
    return _read(filepath_or_buffer, kwds)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 462, in _read
    parser = TextFileReader(filepath_or_buffer, **kwds)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 819, in __init__
    self._engine = self._make_engine(self.engine)
  File "/opt/conda/lib/python3.7/site-packages/pandas/io/parsers.py", line 1050, in _make_engine
    return mapping[engine](self.f, **self.options)  # t

In [10]:
Path(f'{PRE_PATH}').mkdir(parents=True, exist_ok=True)

for code in codes:
    start = time.time()
    try:
        # === user kind merged to hear
        df_dict[code]['user_kind'] = df_dict[code].apply(lambda row: 'anonymous' if pd.isna(row.event_user_id) else 'bot' if not pd.isna(row.event_user_is_bot_by) else 'account', axis=1)
        Logger.instance('pipeline').info(f'User-Kind assignment for {code} in {time.time() - start}')
        
        # === Convert timestamp
        df_dict[code]['event_timestamp_t'] = df_dict[code].event_timestamp.swifter.apply(pd.to_datetime) #pd.to_datetime(df_dict[code].event_timestamp)
        # Set UTC date
        df_dict[code]["event_timestamp_t"] = df_dict[code].event_timestamp_t.dt.tz_localize("UTC", ambiguous='NaT', nonexistent='NaT')
        df_dict[code]["date_utc"] = df_dict[code].event_timestamp_t.dt.strftime("%Y%m%d").astype(int)
        # Localize date
        df_dict[code]["event_timestamp_t"] = df_dict[code].event_timestamp_t.dt.tz_convert(tz_[code])
        df_dict[code]["date"] = df_dict[code].event_timestamp_t.dt.strftime("%Y%m%d").astype(int)
        Logger.instance('pipeline').info(f'Finished date assignment for {code} in {time.time() - start}')
        
        # === Normalize date
        df_dict[code]['page_title_norm'] = df_dict[code].page_title.swifter.apply(lambda title: mw_t.normalize(str(title)) if not pd.isna(title) else np.nan)
        df_dict[code]['page_title_historical_norm'] = df_dict[code].page_title_historical.swifter.apply(lambda title: mw_t.normalize(str(title)) if not pd.isna(title) else np.nan)
        Logger.instance('pipeline').info(f'Finished name normalization for {code} in {time.time() - start}')
    except Exception as e:
        traceback.print_exc()
        Logger.instance('pipeline').info(f'Error for {code}: {str(e)}')

09-01 09:47 : INFO : Created new singleton instance
09-01 09:47 : INFO : User-Kind assignment for pl in 291.5884621143341


New logging instance for ../logging//pipeline.log


/opt/conda/lib/python3.7/site-packages/swifter/swifter.py:37: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  "This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`."
09-01 09:51 : INFO : Finished date assignment for pl in 520.4055099487305


Pandas Apply:   0%|          | 0/12279549 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/swifter/swifter.py:37: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  "This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`."


...

09-01 10:06 : INFO : Finished name normalization for uk in 599.4014620780945
09-01 10:16 : INFO : User-Kind assignment for ru in 583.876383304596
/opt/conda/lib/python3.7/site-packages/swifter/swifter.py:37: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  "This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`."
09-01 10:24 : INFO : Finished date assignment for ru in 1064.6063442230225


Pandas Apply:   0%|          | 0/25160718 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/swifter/swifter.py:37: UserWarning: This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`.
  "This pandas object has duplicate indices, and swifter may not be able to improve performance. Consider resetting the indices with `df.reset_index(drop=True)`."


Pandas Apply:   0%|          | 0/25160718 [00:00<?, ?it/s]

09-01 10:27 : INFO : Finished name normalization for ru in 1265.944834470749


In [ ]:
# OPTIONAL! save preprocessing so far
# NOTE: takes up lots of memory.
# write to processed
for code in codes:
    try:
        start = time.time()
        df_dict[code].to_csv(f'{PRE_PATH}/{code}_mwh_processed.tsv.gz', header=True, index=False, sep="\t", compression='gzip')
        Logger.instance('pipeline').info(f'Dumping {code} done in {time.time() - start}')
    except Exception as e:
        traceback.print_exc()
        Logger.instance('pipeline').info(f'Error when saving {code}: {str(e)}')

### Group By Date and User_Kind to get newcomers

In [11]:
from helpers.files import save_to_pickle

In [12]:
dict_creation = {}
for code in codes:
    start = time.time()
    try:
        df_code = df_dict[code]
        # define masks
        create_event_mask = (df_code.event_entity=='user') & (df_code.event_type == 'create')
        create_revision_mask = (df_code.event_entity=='revision') & (df_code.event_type == 'create')
        no_bot_mask = (df_code['event_user_is_bot_by'].isna() | df_code['event_user_is_bot_by_historical'].isna())
        self_creation_mask = (df_code['event_user_is_created_by_self'] == 'true')
        no_anon_mask = (df_code['event_user_is_anonymous'] != 'true')

        # === get users by registration
        group_creation = df_code[create_event_mask & no_anon_mask & no_bot_mask & self_creation_mask].groupby(['date'])['event_user_id'].size()

        # === get user by nth edit
        # n=1
        edit_count_mask = df_code.event_user_revision_count == '1'
        group_edit1 = df_code[create_revision_mask & no_anon_mask & no_bot_mask & edit_count_mask].groupby(['date'])['event_user_id'].size().rename('edit_1')
        # n=5
        edit_count_mask = df_code.event_user_revision_count == '5'
        group_edit5 = df_code[create_revision_mask & no_anon_mask & no_bot_mask & edit_count_mask].groupby(['date'])['event_user_id'].size().rename('edit_5')
        dict_creation[code] = pd.concat([group_creation, group_edit1, group_edit5], axis=1).fillna(0)
        Logger.instance('pipeline').info(f'User creation computation for {code} done in {time.time() - start}')
    except Exception as e:
        traceback.print_exc()
        Logger.instance('pipeline').info(f'Error for {code}: {str(e)}')
        
save_to_pickle(f'{PRE_PATH}/dict_newcomers_selfcreated_demo.pkl', dict_creation)
Logger.instance('pipeline').info(f'Finished newcomers')

09-01 10:28 : INFO : User creation computation for pl done in 43.03811717033386
09-01 10:28 : INFO : User creation computation for cs done in 13.797757148742676
09-01 10:28 : INFO : User creation computation for uk done in 36.32571029663086
09-01 10:30 : INFO : User creation computation for ru done in 89.68020296096802
09-01 10:30 : INFO : Finished newcomers


### Group By Date and Page and user_kind to get edits per day

In [13]:
# Group By Date and Page and user_kind to get edits per day
dict_edits_byid = {} # grouped by id
dict_edits_bytitle = {} # grouped by title
for code in codes:
    try:
        start = time.time()
        df_code = df_dict[code]
        create_revision_mask = (df_code.event_entity=='revision') & (df_code.event_type == 'create')
        ns_mask = df_code.page_namespace == '0'

        # group by date, page_id, user_kind
        df_code.revision_text_bytes_diff = pd.to_numeric(df_code['revision_text_bytes_diff'], errors='coerce').fillna(0)
        df_code_masked = df_code[create_revision_mask & ns_mask]
        
        dict_edits_byid[code] = df_code_masked.groupby(['date', 'page_id', 'user_kind']).agg(
            {'event_user_id': 'size', 'revision_text_bytes_diff': 'sum', 'page_title': 'last', 
             'page_title_norm': 'last', 'page_title_historical_norm': 'last'})
        dict_edits_bytitle[code] = df_code_masked.groupby(['date', 'page_title_norm', 'user_kind']).agg(
            {'event_user_id': 'size', 'revision_text_bytes_diff': 'sum', 'page_id': lambda x: set(x), 'page_title': lambda x: set(x),  'page_title_historical_norm': lambda x: set(x)})

        Logger.instance('pipeline').info(f'Grouped by user/user_kind/pageid for {code} done in {time.time() - start}')
    except Exception as e:
        traceback.print_exc()
        Logger.instance('pipeline').info(f'Error for {code}: {str(e)}')  
    
save_to_pickle(f'{PRE_PATH}/dict_edits_byid_demo.pkl', dict_edits_byid)
save_to_pickle(f'{PRE_PATH}/dict_edits_bytitle_demo.pkl', dict_edits_bytitle)
Logger.instance('pipeline').info(f'Finished edits')

09-01 10:36 : INFO : Grouped by user/user_kind/pageid for pl done in 387.51079082489014
09-01 10:39 : INFO : Grouped by user/user_kind/pageid for cs done in 128.68407654762268
09-01 10:43 : INFO : Grouped by user/user_kind/pageid for uk done in 275.36093950271606
09-01 10:54 : INFO : Grouped by user/user_kind/pageid for ru done in 640.466635465622
09-01 10:56 : INFO : Finished edits


### Group By Date and user_kind to get identity reverts per day (see above)

In [14]:
dict_reverts = {}
for code in codes:
    try:
        start = time.time()
        df_code = df_dict[code]
        create_revision_mask = (df_code.event_entity=='revision') & (df_code.event_type == 'create')
        ns_mask = df_code.page_namespace == '0'

        # get reverts per day as well as reverted
        df_reverted = df_code[create_revision_mask & ns_mask & (df_code.revision_is_identity_reverted == 'true')].groupby(['date', 'user_kind'])['revision_is_identity_reverted'].size()
        df_reverts = df_code[create_revision_mask & ns_mask & (df_code.revision_is_identity_revert == 'true')].groupby(['date', 'user_kind'])['revision_is_identity_revert'].size()
        
        # reindex so all dates are filled
        df_reverted = df_reverted.reindex(
            pd.MultiIndex.from_product([df_code.date.unique(), df_reverted.index.levels[1]], names=['date', 'user_kind']), fill_value=0)
        df_reverts = df_reverts.reindex(
            pd.MultiIndex.from_product([df_code.date.unique(), df_reverts.index.levels[1]], names=['date', 'user_kind']), fill_value=0)
    
        dict_reverts[code] = pd.concat([df_reverted, df_reverts], axis=1).fillna(0)
        Logger.instance('pipeline').info(f'Computed reverts by {code} done in {time.time() - start}')
    except Exception as e:
        traceback.print_exc()
        Logger.instance('pipeline').info(f'Error for {code}: {str(e)}')  
    
save_to_pickle(f'{PRE_PATH}/dict_reverts_demo.pkl', dict_reverts)
Logger.instance('pipeline').info(f'Finished identity reverts')

09-01 10:56 : INFO : Computed reverts by pl done in 13.750905275344849
09-01 10:56 : INFO : Computed reverts by cs done in 4.97031044960022
09-01 10:56 : INFO : Computed reverts by uk done in 11.530215978622437
09-01 10:57 : INFO : Computed reverts by ru done in 34.912052154541016
09-01 10:57 : INFO : Finished identity reverts


## Combine edit dictionary with covid info

For this simple demo, we exclude covid analysis. However, one could easily declare the articles they want to consider as COVID-19 articles (e.g., retrieving from https://covid-data.wmflabs.org/ or crawling COVID-categories via https://petscan.wmflabs.org/) and merge the result with the given dataframe here.

In [16]:
# This file was pre-generated using the .json-list of COVID-articles from:
# https://covid-data.wmflabs.org/
'''
path_covid = f'{DATA_PATH}/covid_linked.f'
df_covid = pd.read_feather(path_covid)
df_covid['covid'] = True
df_covid['index'] = df_covid['index'].apply(lambda t: mw_t.normalize(str(t)))
'''

In [15]:
'''
df_edits_covid = {}
for code, df_code in dict_edits_bytitle.items():
    df_edits_covid[code] = df_code.reset_index().merge(df_covid[df_covid.site == f'{code}wiki'], left_on=['page_title_norm'], right_on=['index'], how='left').fillna({'covid': False}).drop(['index', 'site', 'qid'], axis=1)

save_to_pickle(f'{PRE_PATH}/dict_edits_bytitle_covid.pkl', df_edits_covid)
'''
for code, df_code in dict_edits_bytitle.items():
    df_code['covid'] = False

# Generate Final Aggregation

In [16]:
from helpers.preprocessing import aggregate_preprocess_results

In [17]:
#final_agg = aggregate_preprocess_results(codes, df_edits_covid, dict_creation, dict_reverts)
final_agg = aggregate_preprocess_results(codes, dict_edits_bytitle, dict_creation, dict_reverts)
final_agg

reading and making sure all dates are filled


09-01 10:57 : INFO : Processing pl took 23.426910638809204


reading and making sure all dates are filled


09-01 10:58 : INFO : Processing cs took 7.83440637588501


reading and making sure all dates are filled


09-01 10:58 : INFO : Processing uk took 17.97576141357422


reading and making sure all dates are filled


09-01 10:59 : INFO : Processing ru took 40.75509834289551


,date,covid,user_kind,count,rev_len_sum,actor_user,edit_1,edit_5,revision_is_identity_reverted,revision_is_identity_revert,code
0,2018-01-01,False,account,4067,1188498.0,54.0,20.0,16.0,134,304,pl
1,2018-01-01,False,anonymous,629,132469.0,0.0,0.0,0.0,219,25,pl
2,2018-01-01,False,bot,560,7606.0,0.0,0.0,0.0,5,2,pl
3,2018-01-02,False,account,4264,1102137.0,168.0,76.0,25.0,143,314,pl
4,2018-01-02,False,anonymous,824,87568.0,0.0,0.0,0.0,331,18,pl
...,...,...,...,...,...,...,...,...,...,...,...
3286,2020-12-31,False,anonymous,2377,186786.0,0.0,0.0,0.0,540,85,ru
3287,2020-12-31,False,bot,1254,342755.0,0.0,0.0,0.0,20,52,ru
3288,2021-01-01,False,account,1078,472596.0,15.0,8.0,4.0,40,48,ru
3289,2021-01-01,False,anonymous,206,21604.0,0.0,0.0,0.0,65,5,ru


In [18]:
Path(f'{RES_PATH}/demo/').mkdir(parents=True, exist_ok=True)
final_agg.to_csv(f'{RES_PATH}/demo/aggregated_demo.tsv.gz', index=False, sep="\t", compression="gzip")